# Incomplete work

### Run experiment on GPU


In [9]:
import os
import numpy as np
import copy
import pickle
import torch
import torch.optim as optim
import random

from data.data_loading import DataLoader, load_dataset, load_graph_dataset
from torch_geometric.data import DataLoader as PyGDataLoader
from exp.train_utils import train, eval, Evaluator
from exp.parser import get_parser, validate_args
from mp.graph_models import GIN0, GINWithJK
from mp.models import CIN0, Dummy, SparseCIN, EdgeOrient, EdgeMPNN, MessagePassingAgnostic
from mp.molec_models import EmbedSparseCIN, OGBEmbedSparseCIN, EmbedSparseCINNoRings, EmbedGIN
from mp.ring_exp_models import RingSparseCIN, RingGIN


def main(args):
    device = torch.device(
        "cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")

    print("======================== Args ===========================")
    print(args)
    print("===================================================")

    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    np.random.seed(args.seed)
    random.seed(args.seed)

    
    if args.model.startswith('gin'):  
        graph_list, train_ids, val_ids, test_ids, num_classes = load_graph_dataset(
            args.dataset, fold=args.fold, max_ring_size=args.max_ring_size)
        train_graphs = [graph_list[i] for i in train_ids]
        val_graphs = [graph_list[i] for i in val_ids]
        train_loader = PyGDataLoader(train_graphs, batch_size=args.batch_size,
                                  shuffle=True, num_workers=args.num_workers)
        valid_loader = PyGDataLoader(val_graphs, batch_size=args.batch_size,
                                   shuffle=False, num_workers=args.num_workers)
        if test_ids is not None:
            test_graphs = [graph_list[i] for i in test_ids]
            test_loader = PyGDataLoader(test_graphs, batch_size=args.batch_size,
                                       shuffle=False, num_workers=args.num_workers)
        else:
            test_loader = None
        if args.dataset.startswith('sr'):
            num_features = 1
            num_classes = args.emb_dim
        else:
            num_features = graph_list[0].x.shape[1]

    else:
        dataset = load_dataset(args.dataset, max_dim=args.max_dim, fold=args.fold,
                               init_method=args.init_method, emb_dim=args.emb_dim,
                               flow_points=args.flow_points, flow_classes=args.flow_classes,
                               max_ring_size=args.max_ring_size,
                               use_edge_features=args.use_edge_features,
                               simple_features=args.simple_features, n_jobs=args.preproc_jobs,
                               train_orient=args.train_orient, test_orient=args.test_orient)
        if args.tune:
            split_idx = dataset.get_tune_idx_split()
        else:
            split_idx = dataset.get_idx_split()

        train_loader = DataLoader(dataset.get_split('train'), batch_size=args.batch_size,
            shuffle=True, num_workers=args.num_workers, max_dim=dataset.max_dim)
        valid_loader = DataLoader(dataset.get_split('valid'), batch_size=args.batch_size,
            shuffle=False, num_workers=args.num_workers, max_dim=dataset.max_dim)
        test_split = split_idx.get("test", None)
        test_loader = None
        if test_split is not None:
            test_loader = DataLoader(dataset.get_split('test'), batch_size=args.batch_size,
                shuffle=False, num_workers=args.num_workers, max_dim=dataset.max_dim)

    evaluator = Evaluator(args.eval_metric)

    use_coboundaries = args.use_coboundaries.lower() == 'true'

    readout_dims = tuple(sorted(args.readout_dims))

    if args.model == 'cin':
        model = CIN0(dataset.num_features_in_dim(0),          # num_input_features
                     dataset.num_classes,                     # num_classes
                     args.num_layers,                         # num_layers
                     args.emb_dim,                            # hidden
                     dropout_rate=args.drop_rate,             # dropout rate
                     max_dim=dataset.max_dim,                 # max_dim
                     jump_mode=args.jump_mode,                # jump mode
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     readout=args.readout,                    # readout
                    ).to(device)
    elif args.model == 'sparse_cin':
        model = SparseCIN(dataset.num_features_in_dim(0),     # num_input_features
                     dataset.num_classes,                     # num_classes
                     args.num_layers,                         # num_layers
                     args.emb_dim,                            # hidden
                     dropout_rate=args.drop_rate,             # dropout rate
                     max_dim=dataset.max_dim,                 # max_dim
                     jump_mode=args.jump_mode,                # jump mode
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     readout=args.readout,                    # readout
                     final_readout=args.final_readout,        # final readout
                     apply_dropout_before=args.drop_position, # where to apply dropout
                     use_coboundaries=use_coboundaries,       # whether to use coboundaries in up-msg
                     graph_norm=args.graph_norm,              # normalization layer
                     readout_dims=readout_dims                # readout_dims
                    ).to(device)
    elif args.model == 'ring_sparse_cin':
        model = RingSparseCIN(
                     dataset.num_features_in_dim(0),          # num_input_features
                     dataset.num_classes,                     # num_classes
                     args.num_layers,                         # num_layers
                     args.emb_dim,                            # hidden
                     max_dim=dataset.max_dim,                 # max_dim
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     use_coboundaries=use_coboundaries,       # whether to use coboundaries in up-msg
                     graph_norm=args.graph_norm,              # normalization layer
                    ).to(device)
    elif args.model == 'gin':
        model = GIN0(num_features,                            # num_input_features
                     args.num_layers,                         # num_layers
                     args.emb_dim,                            # hidden
                     num_classes,                             # num_classes
                     dropout_rate=args.drop_rate,             # dropout rate
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     readout=args.readout,                    # readout
        ).to(device)
    elif args.model == 'gin_ring':
        model = RingGIN(num_features,                                # num_input_features
                            args.num_layers,                         # num_layers
                            args.emb_dim,                            # hidden
                            num_classes,                             # num_classes
                            nonlinearity=args.nonlinearity,          # nonlinearity
                            graph_norm=args.graph_norm,              # normalization layer
        ).to(device)
    elif args.model == 'gin_jk':
        model = GINWithJK(num_features,                       # num_input_features
                     args.num_layers,                         # num_layers
                     args.emb_dim,                            # hidden
                     num_classes,                             # num_classes
                     dropout_rate=args.drop_rate,             # dropout rate
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     readout=args.readout,                    # readout
        ).to(device)
    elif args.model == 'mp_agnostic':
        model = MessagePassingAgnostic(
                     dataset.num_features_in_dim(0),          # num_input_features
                     dataset.num_classes,                     # num_classes
                     args.emb_dim,                            # hidden
                     dropout_rate=args.drop_rate,             # dropout rate
                     max_dim=dataset.max_dim,                 # max_dim
                     nonlinearity=args.nonlinearity,          # nonlinearity
                     readout=args.readout,                    # readout
                    ).to(device)
    elif args.model == 'dummy':
        model = Dummy(dataset.num_features_in_dim(0),
                      dataset.num_classes,
                      args.num_layers,
                      max_dim=dataset.max_dim,
                      readout=args.readout,
                     ).to(device)
    elif args.model == 'edge_orient':
        model = EdgeOrient(1,
                      dataset.num_classes,
                      args.num_layers,
                      args.emb_dim,  # hidden
                      readout=args.readout,
                      nonlinearity=args.nonlinearity,  # nonlinearity
                      dropout_rate=args.drop_rate,  # dropout rate
                      fully_invar=args.fully_orient_invar
        ).to(device)
    elif args.model == 'edge_mpnn':
        model = EdgeMPNN(1,
                      dataset.num_classes,
                      args.num_layers,
                      args.emb_dim,  # hidden
                      readout=args.readout,
                      nonlinearity=args.nonlinearity,  # nonlinearity
                      dropout_rate=args.drop_rate,  # dropout rate
                      fully_invar=args.fully_orient_invar,
        ).to(device)
    elif args.model == 'embed_sparse_cin':
        model = EmbedSparseCIN(dataset.num_node_type,  # The number of atomic types
                               dataset.num_edge_type,  # The number of bond types
                               dataset.num_classes,  # num_classes
                               args.num_layers,  # num_layers
                               args.emb_dim,  # hidden
                               dropout_rate=args.drop_rate,  # dropout rate
                               max_dim=dataset.max_dim,  # max_dim
                               jump_mode=args.jump_mode,  # jump mode
                               nonlinearity=args.nonlinearity,  # nonlinearity
                               readout=args.readout,  # readout
                               final_readout=args.final_readout,  # final readout
                               apply_dropout_before=args.drop_position,  # where to apply dropout
                               use_coboundaries=use_coboundaries,
                               embed_edge=args.use_edge_features,
                               graph_norm=args.graph_norm,  # normalization layer
                               readout_dims=readout_dims  # readout_dims
                               ).to(device)
    elif args.model == 'embed_sparse_cin_no_rings':
        model = EmbedSparseCINNoRings(dataset.num_node_type,  # The number of atomic types
                                      dataset.num_edge_type,  # The number of bond types
                                      dataset.num_classes,  # num_classes
                                      args.num_layers,  # num_layers
                                      args.emb_dim,  # hidden
                                      dropout_rate=args.drop_rate,  # dropout rate
                                      nonlinearity=args.nonlinearity,  # nonlinearity
                                      readout=args.readout,  # readout
                                      final_readout=args.final_readout,  # final readout
                                      apply_dropout_before=args.drop_position,  # where to apply dropout
                                      use_coboundaries=use_coboundaries,
                                      embed_edge=args.use_edge_features,
                                      graph_norm=args.graph_norm,  # normalization layer
        ).to(device)
    elif args.model == 'embed_gin':
        model = EmbedGIN(dataset.num_node_type,  # The number of atomic types
                         dataset.num_edge_type,  # The number of bond types
                         dataset.num_classes,  # num_classes
                         args.num_layers,  # num_layers
                         args.emb_dim,  # hidden
                         dropout_rate=args.drop_rate,  # dropout rate
                         nonlinearity=args.nonlinearity,  # nonlinearity
                         readout=args.readout,  # readout
                         apply_dropout_before=args.drop_position,  # where to apply dropout
                         embed_edge=args.use_edge_features,
        ).to(device)
    elif args.model == 'ogb_embed_sparse_cin':
        model = OGBEmbedSparseCIN(dataset.num_tasks,                       # out_size
                                  args.num_layers,                         # num_layers
                                  args.emb_dim,                            # hidden
                                  dropout_rate=args.drop_rate,             # dropout_rate
                                  indropout_rate=args.indrop_rate,         # in-dropout_rate
                                  max_dim=dataset.max_dim,                 # max_dim
                                  jump_mode=args.jump_mode,                # jump_mode
                                  nonlinearity=args.nonlinearity,          # nonlinearity
                                  readout=args.readout,                    # readout
                                  final_readout=args.final_readout,        # final readout
                                  apply_dropout_before=args.drop_position, # where to apply dropout
                                  use_coboundaries=use_coboundaries,       # whether to use coboundaries
                                  embed_edge=args.use_edge_features,       # whether to use edge feats
                                  graph_norm=args.graph_norm,              # normalization layer
                                  readout_dims=readout_dims                # readout_dims
                                 ).to(device)
    else:
        raise ValueError('Invalid model type {}.'.format(args.model))

    print("============= Model Parameters =================")
    trainable_params = 0
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(name, param.size())
            trainable_params += param.numel()
        total_params += param.numel()
    print("============= Params stats ==================")
    print(f"Trainable params: {trainable_params}")
    print(f"Total params    : {total_params}")

    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.lr_scheduler == 'ReduceLROnPlateau':
        mode = 'min' if args.minimize else 'max'
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode=mode,
                                                               factor=args.lr_scheduler_decay_rate,
                                                               patience=args.lr_scheduler_patience,
                                                               verbose=True)
    elif args.lr_scheduler == 'StepLR':
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_scheduler_decay_steps,
                                                    gamma=args.lr_scheduler_decay_rate)
    elif args.lr_scheduler == 'None':
        scheduler = None
    else:
        raise NotImplementedError(f'Scheduler {args.lr_scheduler} is not currently supported.')

    best_val_epoch = 0
    valid_curve = []
    test_curve = []
    train_curve = []
    train_loss_curve = []
    params = []
    if not args.untrained:
        for epoch in range(1, args.epochs + 1):

            print("=====Epoch {}".format(epoch))
            print('Training...')
            epoch_train_curve = train(model, device, train_loader, optimizer, args.task_type)
            train_loss_curve += epoch_train_curve
            epoch_train_loss = float(np.mean(epoch_train_curve))

            print('Evaluating...')
            if epoch == 1 or epoch % args.train_eval_period == 0:
                train_perf, _ = eval(model, device, train_loader, evaluator, args.task_type)
            train_curve.append(train_perf)
            valid_perf, epoch_val_loss = eval(model, device,
                valid_loader, evaluator, args.task_type)
            valid_curve.append(valid_perf)

            if test_loader is not None:
                test_perf, epoch_test_loss = eval(model, device, test_loader, evaluator,
                                                  args.task_type)
            else:
                test_perf = np.nan
                epoch_test_loss = np.nan
            test_curve.append(test_perf)

            print(f'Train: {train_perf:.3f} | Validation: {valid_perf:.3f} | Test: {test_perf:.3f}'
                  f' | Train Loss {epoch_train_loss:.3f} | Val Loss {epoch_val_loss:.3f}'
                  f' | Test Loss {epoch_test_loss:.3f}')

            if scheduler is not None:
                if args.lr_scheduler == 'ReduceLROnPlateau':
                    scheduler.step(valid_perf)
                    if args.early_stop and optimizer.param_groups[0]['lr'] < args.lr_scheduler_min:
                        print("\n!! The minimum learning rate has been reached.")
                        break
                else:
                    scheduler.step()

            i = 0
            new_params = []
            if epoch % args.train_eval_period == 0:
                print("====== Slowly changing params ======= ")
            for name, param in model.named_parameters():
                new_params.append(param.data.detach().mean().item())
                if len(params) > 0 and epoch % args.train_eval_period == 0:
                    if abs(params[i] - new_params[i]) < 1e-6:
                        print(f"Param {name}: {params[i] - new_params[i]}")
                i += 1
            params = copy.copy(new_params)

        if not args.minimize:
            best_val_epoch = np.argmax(np.array(valid_curve))
        else:
            best_val_epoch = np.argmin(np.array(valid_curve))
    else:
        train_loss_curve.append(np.nan)
        train_curve.append(np.nan)
        valid_curve.append(np.nan)
        test_curve.append(np.nan)

    print('Final Evaluation...')
    final_train_perf, _ = eval(model, device, train_loader, evaluator, args.task_type)
    final_val_perf, _ = eval(model, device, valid_loader, evaluator, args.task_type)
    final_test_perf = np.nan
    if test_loader is not None:
        final_test_perf, _ = eval(model, device, test_loader, evaluator, args.task_type)

    curves = {
        'train_loss': train_loss_curve,
        'train': train_curve,
        'val': valid_curve,
        'test': test_curve,
        'last_val': final_val_perf,
        'last_test': final_test_perf,
        'last_train': final_train_perf,
        'best': best_val_epoch}

    msg = (
       f'========== Result ============\n'
       f'Dataset:        {args.dataset}\n'
       f'------------ Best epoch -----------\n'
       f'Train:          {train_curve[best_val_epoch]}\n'
       f'Validation:     {valid_curve[best_val_epoch]}\n'
       f'Test:           {test_curve[best_val_epoch]}\n'
       f'Best epoch:     {best_val_epoch}\n'
       '------------ Last epoch -----------\n'
       f'Train:          {final_train_perf}\n'
       f'Validation:     {final_val_perf}\n'
       f'Test:           {final_test_perf}\n'
       '-------------------------------\n\n')
    print(msg)

    msg += str(args)
    with open(filename, 'w') as handle:
        handle.write(msg)
    if args.dump_curves:
        with open(os.path.join(result_folder, 'curves.pkl'), 'wb') as handle:
            pickle.dump(curves, handle)

    return curves


if __name__ == "__main__":

    parser = get_parser()
    args = parser.parse_args()
    validate_args(args)

    main(args)


usage: ipykernel_launcher.py [-h] [--seed SEED] [--start_seed START_SEED]
                             [--stop_seed STOP_SEED] [--device DEVICE]
                             [--model MODEL]
                             [--use_coboundaries USE_COBOUNDARIES]
                             [--indrop_rate INDROP_RATE]
                             [--drop_rate DROP_RATE]
                             [--drop_position DROP_POSITION]
                             [--nonlinearity NONLINEARITY] [--readout READOUT]
                             [--final_readout FINAL_READOUT]
                             [--readout_dims READOUT_DIMS [READOUT_DIMS ...]]
                             [--jump_mode JUMP_MODE] [--graph_norm {bn,ln,id}]
                             [--lr LR] [--lr_scheduler LR_SCHEDULER]
                             [--lr_scheduler_decay_steps LR_SCHEDULER_DECAY_STEPS]
                             [--lr_scheduler_decay_rate LR_SCHEDULER_DECAY_RATE]
                             [--lr_schedul

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**The conda environment is already set up on colab so I just need to clone the repo and do my experiments according.**

In [7]:
!pwd

/content/cwn


In [3]:
# clone the repo
!git clone https://github.com/twitter-research/cwn.git

Cloning into 'cwn'...
remote: Enumerating objects: 4251, done.
remote: Counting objects: 100% (2493/2493), done.
remote: Compressing objects: 100% (507/507), done.
remote: Total 4251 (delta 2096), reused 2247 (delta 1974), pack-reused 1758
Receiving objects: 100% (4251/4251), 69.70 MiB | 33.07 MiB/s, done.
Resolving deltas: 100% (3104/3104), done.


In [4]:
# change directory to the cwn folder
%cd cwn

/content/cwn


# The four lines for the dependencies are equivalent to the following on colab.

In [5]:
# first line that corrseponds to conda
!pip install torch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0

     |████████████████████████████████| 776.7 MB 4.3 kB/s 
     |████████████████████████████████| 11.8 MB 30.4 MB/s 
     |████████████████████████████████| 7.6 MB 25.5 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.


In [6]:
# second line of code for the dependencies
!sh pyG_install.sh cu102

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
     |████████████████████████████████| 12.0 MB 1.1 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
     |████████████████████████████████| 24.3 MB 68 kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
     |████████████████████████████████| 21.5 MB 130 kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu102.html
     |████████████████████████████████| 6.4 MB 7.5 kB/s 
     |████████████████████████████████| 186 kB 8.8 MB/s 
     |████████████████████████████████| 376 kB 58.1 MB/s 
     |████████████████████████████████| 2.2 MB 52.1 MB/s 
     |████████████████████████████████| 45 kB 4.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-py3-none-any.whl size=322733 sha256=7a4566a1caba59de6a351ca46f2ee0d8c1c88b20a278b667933eee7fc7dfce6a
  Stored in directory: /root/.cache/pip/wheels/9f/98/e2/5e1e57b8ac9f1354

In [7]:
# third line of code for the dependencies
!pip install -r requirements.txt

     |████████████████████████████████| 279 kB 6.9 MB/s 
     |████████████████████████████████| 26.1 MB 2.6 kB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
     |████████████████████████████████| 375 kB 63.8 MB/s 
     |████████████████████████████████| 11.6 MB 57.5 MB/s 
     |████████████████████████████████| 302 kB 58.3 MB/s 
     |████████████████████████████████| 1.6 MB 35.2 MB/s 
     |████████████████████████████████| 14.5 MB 75 kB/s 
     |████████████████████████████████| 69 kB 8.9 MB/s 
     |████████████████████████████████| 6.8 MB 61.9 MB/s 
     |████████████████████████████████| 25.9 MB 1.0 kB/s 
  Created wheel for pyyaml: filename=PyYAML-3.12-cp37-cp37m-linux_x86_64.whl size=43073 sha256=6fe4a51184940108c55d4a375e947a88407be971f265aa9cadb489eeeef378d9
  Stored in directory: /root/.cache/pip/wheels/21/cc/0a/8f691c938b079b07fa17a2e2940e3faae0a3dfb3ff5b3ac356
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=561474

# Restart runtime and start running the code from the cell below


In [1]:
!pwd

/content


In [2]:
%cd cwn

/content/cwn


In [3]:
# fourth line of code for the dependencies

!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib

Executing: /tmp/apt-key-gpghome.pc2EmeiXeR/gpg.1.sh --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4

In [ ]:
#!pytest -v .

# The Experiment

## To run an experiment on the SR benchmark with a CWN, run:
#### replacing <k> with a value amongst 4, 5, 6 (<k> is the maximum ring size employed in the lifting procedure). The results, for each family, will be written under exp/results/SR-cwn-sr-<k>/.

The following command will run the MLP-sum (strong) baseline on the same ring-lifted graphs:

In [ ]:
!sh exp/scripts/cwn-sr.sh 4

[i] family sr16622, seed 0
Using device cuda:0
Fold: None
Seed: 0
======================== Args ===========================
Namespace(batch_size=8, dataset='sr16622', device=0, drop_position='lin2', drop_rate=0.0, dump_curves=False, early_stop=False, emb_dim=16, epochs=100, eval_metric='isomorphism', exp_name='cwn-sr', final_readout='sum', flow_classes=3, flow_points=400, fold=None, folds=None, fully_orient_invar=False, graph_norm='id', indrop_rate=0.0, init_method='sum', jump_mode=None, lr=0.001, lr_scheduler='None', lr_scheduler_decay_rate=0.5, lr_scheduler_decay_steps=50, lr_scheduler_min=1e-05, lr_scheduler_patience=10, max_dim=2, max_ring_size=4, minimize=False, model='sparse_cin', nonlinearity='elu', num_layers=5, num_workers=2, paraid=0, preproc_jobs=64, readout='sum', readout_dims=[0, 1, 2], result_folder='/content/cwn/exp/results/SR-cwn-sr-4', seed=0, simple_features=False, start_seed=0, stop_seed=4, task_type='isomorphism', test_orient='default', train_eval_period=10, train_o

In [4]:
!sh exp/scripts/cwn-sr.sh 5

[i] family sr16622, seed 0
Using device cuda:0
Fold: None
Seed: 0
======================== Args ===========================
Namespace(batch_size=8, dataset='sr16622', device=0, drop_position='lin2', drop_rate=0.0, dump_curves=False, early_stop=False, emb_dim=16, epochs=100, eval_metric='isomorphism', exp_name='cwn-sr', final_readout='sum', flow_classes=3, flow_points=400, fold=None, folds=None, fully_orient_invar=False, graph_norm='id', indrop_rate=0.0, init_method='sum', jump_mode=None, lr=0.001, lr_scheduler='None', lr_scheduler_decay_rate=0.5, lr_scheduler_decay_steps=50, lr_scheduler_min=1e-05, lr_scheduler_patience=10, max_dim=2, max_ring_size=5, minimize=False, model='sparse_cin', nonlinearity='elu', num_layers=5, num_workers=2, paraid=0, preproc_jobs=64, readout='sum', readout_dims=[0, 1, 2], result_folder='/content/cwn/exp/results/SR-cwn-sr-5', seed=0, simple_features=False, start_seed=0, stop_seed=4, task_type='isomorphism', test_orient='default', train_eval_period=10, train_o

In [ ]:
#!sh exp/scripts/cwn-sr.sh 6

#### The following command will run the MLP-sum (strong) baseline on the same ring-lifted graphs:

In [ ]:
!sh exp/scripts/cwn-sr-base.sh 4

In [ ]:
!sh exp/scripts/cwn-sr-base.sh 5

In [ ]:
!sh exp/scripts/cwn-sr-base.sh 6

#### In order to run these experiment with clique-complex lifting (MPSNs), run:


In [ ]:
#MPSN
!sh exp/scripts/mpsn-sr.sh

Streaming output truncated to the last 5000 lines.
convs.1.mp_levels.3.combine_nn.0.weight torch.Size([16, 32])
convs.1.mp_levels.3.combine_nn.0.bias torch.Size([16])
convs.2.mp_levels.0.msg_up_nn.1.weight torch.Size([16, 32])
convs.2.mp_levels.0.msg_up_nn.1.bias torch.Size([16])
convs.2.mp_levels.0.update_up_nn.0.weight torch.Size([16, 16])
convs.2.mp_levels.0.update_up_nn.0.bias torch.Size([16])
convs.2.mp_levels.0.update_up_nn.3.weight torch.Size([16, 16])
convs.2.mp_levels.0.update_up_nn.3.bias torch.Size([16])
convs.2.mp_levels.0.update_boundaries_nn.0.weight torch.Size([16, 16])
convs.2.mp_levels.0.update_boundaries_nn.0.bias torch.Size([16])
convs.2.mp_levels.0.update_boundaries_nn.3.weight torch.Size([16, 16])
convs.2.mp_levels.0.update_boundaries_nn.3.bias torch.Size([16])
convs.2.mp_levels.0.combine_nn.0.weight torch.Size([16, 32])
convs.2.mp_levels.0.combine_nn.0.bias torch.Size([16])
convs.2.mp_levels.1.msg_up_nn.1.weight torch.Size([16, 32])
convs.2.mp_levels.1.msg_up_nn.1

#### Clique-lifting is applied up to dimension k-1, with k the maximum clique-size in the family.

The MLP-sum baseline on clique-complexes is run with:

In [ ]:
###########
!sh exp/scripts/mpsn-sr-base.sh

sh: 0: Can't open exp/scripts/mpsn-sr-base.sh


In [ ]:
!pwd

/content


In [ ]:
%cd cwn

[Errno 2] No such file or directory: 'cwn'
/content


In [ ]:
!sh exp/scripts/cwn-sr.sh 4

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/cwn/exp/run_sr_exp.py", line 10, in <module>
    from exp.run_exp import main
  File "/content/cwn/exp/run_exp.py", line 9, in <module>
    from data.data_loading import DataLoader, load_dataset, load_graph_dataset
  File "/content/cwn/data/data_loading.py", line 30, in <module>
    from torch_geometric.data import Data, Batch
ModuleNotFoundError: No module named 'torch_geometric'


Streaming output truncated to the last 5000 lines.
convs.2.mp_levels.1.combine_nn.0.weight torch.Size([16, 32])
convs.2.mp_levels.1.combine_nn.0.bias torch.Size([16])
convs.2.mp_levels.2.msg_up_nn.1.weight torch.Size([16, 32])
convs.2.mp_levels.2.msg_up_nn.1.bias torch.Size([16])
convs.2.mp_levels.2.update_up_nn.0.weight torch.Size([16, 16])
convs.2.mp_levels.2.update_up_nn.0.bias torch.Size([16])
convs.2.mp_levels.2.update_up_nn.3.weight torch.Size([16, 16])
convs.2.mp_levels.2.update_up_nn.3.bias torch.Size([16])
convs.2.mp_levels.2.update_boundaries_nn.0.weight torch.Size([16, 16])
convs.2.mp_levels.2.update_boundaries_nn.0.bias torch.Size([16])
convs.2.mp_levels.2.update_boundaries_nn.3.weight torch.Size([16, 16])
convs.2.mp_levels.2.update_boundaries_nn.3.bias torch.Size([16])
convs.2.mp_levels.2.combine_nn.0.weight torch.Size([16, 32])
convs.2.mp_levels.2.combine_nn.0.bias torch.Size([16])
convs.2.mp_levels.3.msg_up_nn.1.weight torch.Size([16, 32])
convs.2.mp_levels.3.msg_up_nn.1

## Results

In [ ]:
!sh exp/scripts/cwn-sr.sh 5

In [ ]:
!sh exp/scripts/cwn-sr.sh 6